In [0]:
source_path = "path_to_silver"
destination_path = "path_to_gold"

In [0]:
dbutils.fs.ls(f"{source_path}") 

In [0]:

df = spark.read.format('delta').load(f'{source_path}/Address/')

In [0]:
display(df.limit(5))

AddressID,AddressLine1,AddressLine2,City,StateProvince,CountryRegion,PostalCode,rowguid,ModifiedDate
9,8713 Yosemite Ct.,null,Bothell,Washington,United States,98011,268af621-76d7-4c78-9441-144fd139821a,2006-07-01
11,1318 Lasalle Street,null,Bothell,Washington,United States,98011,981b3303-aca2-49c7-9a96-fb670785b269,2007-04-01
25,9178 Jumping St.,null,Dallas,Texas,United States,75201,c8df3bd9-48f0-4654-a8dd-14a67a84d3c6,2006-09-01
28,9228 Via Del Sol,null,Phoenix,Arizona,United States,85004,12ae5ee1-fc3e-468b-9b92-3b970b169774,2005-09-01
32,26910 Indela Road,null,Montreal,Quebec,Canada,H1Y 2H5,84a95f62-3ae8-4e7e-bbd5-5a6f00cd982d,2006-08-01


In [0]:
from pyspark.sql import DataFrame

def rename_columns_to_snake_case(df: DataFrame) -> DataFrame:
    # Get the list of column names
    column_names = df.columns

    # Dictionary to hold old and new column name mappings
    rename_map = {}

    for old_col_name in column_names:
        # Convert column name from PascalCase or camelCase to snake_case
        new_col_name = "".join([
            "_" + char.lower() if (
                char.isupper()              # Check if the current character is uppercase
                and idx > 0                 # Ensure it's not the first character
                and not old_col_name[idx - 1].isupper()  # Ensure the previous character is not uppercase
            ) else char.lower()  # Convert character to lowercase
            for idx, char in enumerate(old_col_name)
        ]).lstrip("_")  # Remove any leading underscore

        # Avoid renaming to an existing column name
        if new_col_name in rename_map.values():
            raise ValueError(f"Duplicate column name found after renaming: '{new_col_name}'")

        # Map the old column name to the new column name
        rename_map[old_col_name] = new_col_name

    # Rename columns using the mapping
    for old_col_name, new_col_name in rename_map.items():
        df = df.withColumnRenamed(old_col_name, new_col_name)

    return df


In [0]:
df.columns

['address_id',
 'address_line1',
 'address_line2',
 'city',
 'state_province',
 'country_region',
 'postal_code',
 'rowguid',
 'modified_date']

All table columns transformation

In [0]:
table_name = []
for i in dbutils.fs.ls(f'{source_path}'):
    table_name.append(i.name.split('/')[0])

In [0]:
for name in table_name:
    path = f'{source_path}' + name
    df = spark.read.format('delta').load(path)
    df = rename_columns_to_snake_case(df)

    output_path = f'{destination_path}' + name + '/'
    df.write.format('delta').mode('overwrite').save(output_path)

In [0]:
display(df.limit(10))

sales_order_id,revision_number,order_date,due_date,ship_date,status,online_order_flag,sales_order_number,purchase_order_number,account_number,customer_id,ship_to_address_id,bill_to_address_id,ship_method,credit_card_approval_code,sub_total,tax_amt,freight,total_due,comment,rowguid,modified_date
71774,2,2008-06-01,2008-06-13,2008-06-08,5,false,SO71774,PO348186287,10-4020-000609,29847,1092,1092,CARGO TRANSPORT 5,null,880.3484,70.4279,22.0087,972.7850,null,89e42cdc-8506-48a2-b89b-eb3e64e3554e,2008-06-08
71776,2,2008-06-01,2008-06-13,2008-06-08,5,false,SO71776,PO19952192051,10-4020-000106,30072,640,640,CARGO TRANSPORT 5,null,78.8100,6.3048,1.9703,87.0851,null,8a3448c5-e677-4158-a29b-dd33069be0b0,2008-06-08
71780,2,2008-06-01,2008-06-13,2008-06-08,5,false,SO71780,PO19604173239,10-4020-000340,30113,653,653,CARGO TRANSPORT 5,null,38418.6895,3073.4952,960.4672,42452.6519,null,a47665d2-7ac9-4cf3-8a8b-2a3883554284,2008-06-08
71782,2,2008-06-01,2008-06-13,2008-06-08,5,false,SO71782,PO19372114749,10-4020-000582,29485,1086,1086,CARGO TRANSPORT 5,null,39785.3304,3182.8264,994.6333,43962.7901,null,f1be45a5-5c57-4a50-93c6-5f8be44cb7cb,2008-06-08
71783,2,2008-06-01,2008-06-13,2008-06-08,5,false,SO71783,PO19343113609,10-4020-000024,29957,992,992,CARGO TRANSPORT 5,null,83858.4261,6708.6741,2096.4607,92663.5609,null,7db2329e-6446-42a8-8915-9c8370b68ed8,2008-06-08
71784,2,2008-06-01,2008-06-13,2008-06-08,5,false,SO71784,PO19285135919,10-4020-000448,29736,659,659,CARGO TRANSPORT 5,null,108561.8317,8684.9465,2714.0458,119960.8240,null,ca31f324-2c32-4f8d-95eb-596e7f343027,2008-06-08
71796,2,2008-06-01,2008-06-13,2008-06-08,5,false,SO71796,PO17052159664,10-4020-000420,29660,1058,1058,CARGO TRANSPORT 5,null,57634.6342,4610.7707,1440.8659,63686.2708,null,917ef5ba-f32d-4563-8588-66db0bcdc846,2008-06-08
71797,2,2008-06-01,2008-06-13,2008-06-08,5,false,SO71797,PO16501134889,10-4020-000142,29796,642,642,CARGO TRANSPORT 5,null,78029.6898,6242.3752,1950.7422,86222.8072,null,bb3fee84-c8bf-4dd2-bcca-675ab6a11c38,2008-06-08
71815,2,2008-06-01,2008-06-13,2008-06-08,5,false,SO71815,PO13021155785,10-4020-000276,30089,1034,1034,CARGO TRANSPORT 5,null,1141.5782,91.3263,28.5395,1261.4440,null,2aa5f39b-1096-4a4b-b17b-f10504a397ce,2008-06-08
71816,2,2008-06-01,2008-06-13,2008-06-08,5,false,SO71816,PO12992180445,10-4020-000295,30027,1038,1038,CARGO TRANSPORT 5,null,3398.1659,271.8533,84.9541,3754.9733,null,e3c189e7-98de-4c40-b6c2-0d1d13f9bb33,2008-06-08
